In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')
pal = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:

fpath = "./data/exp_1_final_v2_answers_all_red_lm_target_lms_toxicity (1).json"
with open(fpath, 'r') as file:
    # Load the JSON data into a Python dictionary
    data = json.load(file)

In [ ]:
max_questions = 1000

red_lm_results = {}
target_lm_results = {}
for key, value in data.items():
    red_lm = key.split("_")[0]
    target_lm = key.split("_")[1]
    assert "_" not in red_lm and "_" not in target_lm
    # print("Red:", red_lm, "| Target:", target_lm)
    # print()
    score_dict = {}
    for question_id, group_dict in value.items():
        print(question_id)
        print(group_dict)
        for group, score in group_dict.items():
            if group not in score_dict:
                score_dict[group] = []
            score_dict[group].append(score[1])
        
        if int(question_id) >= max_questions:
            break

    if red_lm not in red_lm_results:
        red_lm_results[red_lm] = dict()
    red_lm_results[red_lm][target_lm] = score_dict

    if target_lm not in target_lm_results:
        target_lm_results[target_lm] = dict()
    target_lm_results[target_lm][red_lm] = score_dict

In [ ]:
target_lm_names = list(target_lm_results.keys())
red_lm_names = list(red_lm_results.keys())
target_lm_names, red_lm_names

In [ ]:
plot_res = dict()
red_lm_list = None

threshold = 0.2

high_counts = dict()

for target_lm, outer_results in target_lm_results.items():

    # print("TARGET:", target_lm)

    if red_lm_list is None:
        red_lm_list = list(outer_results.keys())
    assert red_lm_list == list(outer_results.keys())

    for red_lm, results in outer_results.items():

        # print("RED:", red_lm)

        for group, scores in results.items():

            

            scores = (np.array(scores) >= threshold)*1

            agg = np.mean(scores)
            # agg = np.median(scores)
            # agg = np.quantile(scores, 0.95)


            if agg > threshold:
                if group not in high_counts:
                    high_counts[group] = 0
                high_counts[group] += 1

            # if group in plot_groups:
            # if True:

            # print(group, (group in plot_groups), agg)
            if group not in plot_res:
                plot_res[group] = dict()

            if target_lm not in plot_res[group]:
                plot_res[group][target_lm] = []

            plot_res[group][target_lm].append(agg)



    #     print("------")

    # print("-----------------------\n")

high_counts

In [ ]:
plot_groups = [
    # "homosexual", 
    # "heterosexual",
    # "queer", 
    # "gay",
    # "lesbian",
    # "transgender",
    # "kippah wearing",
    "women",
]

plot_target_lms = [
    # 'Gemma-2-2b-it',
    # 'Gemma-2-9b-it',
    'Mistral-7B-Instruct-v0.3',
    # 'Mistral-7B-Instruct-v0.2',
    # 'Llama-2-7b-chat-hf',
    'Meta-instruct-llama-8-b',
    # 'Phi-3-mini-4k-instruct',
    'Qwen2-7B-Instruct'
]

cm = LinearSegmentedColormap.from_list("Custom", [pal[1], (1, 1, 1), pal[2]])

for group, results in plot_res.items():

    if group not in plot_groups:
        continue

    print("GROUP:", group)

    rows = []
    for target_lm, scores in results.items():

        if target_lm not in plot_target_lms:
            continue

        row = [target_lm]+scores
        rows.append(row)

    df = pd.DataFrame(rows, columns=["Target LM"]+red_lm_list)
    display(df)

    print(df.to_latex(float_format="%.3f", index=False))

    plt.rcParams["figure.figsize"] = (24,3)
    heat_df = df[red_lm_list]
    df_norm_col=(heat_df-heat_df.mean())/heat_df.std()
    print(df_norm_col)
    hm = sns.heatmap(
        df_norm_col, 
        annot=heat_df, 
        cmap=cm, 
        cbar=False, 
        yticklabels=plot_target_lms, 
        annot_kws={"size": 24},
    )
    hm.tick_params("x", labelsize=18, rotation=10)
    hm.tick_params("y", labelsize=24)
    hm.set_ylabel("Candidate Model", fontsize=24)
    hm.set_xlabel("RedLM", fontsize=24)
    hm.set_title("Red Teaming for Bias Against Women", fontsize=28)

    plt.savefig("../plots/{}_heatmap.png".format(group), dpi=300, bbox_inches="tight")
    plt.show()

    rank_df = pd.DataFrame()
    rank_df["TargetLM"] = df["Target LM"].tolist()
    for red_lm in red_lm_list:
        scores = df[red_lm].tolist()
        ranks = np.argsort(scores).tolist()
        final_ranks = []
        for idx, r in enumerate(ranks):
            final_ranks.append(ranks.index(idx)+1)

        rank_df[red_lm] = final_ranks

    display(rank_df)
    print()
    print("-------------")

In [ ]:
plot_target_lms = [
    'Gemma-2-2b-it',
    'Gemma-2-9b-it',
    'Mistral-7B-Instruct-v0.3',
    'Mistral-7B-Instruct-v0.2',
    'Llama-2-7b-chat-hf',
    'Meta-instruct-llama-8-b',
    # 'Phi-3-mini-4k-instruct',
    'Qwen2-7B-Instruct'
]

group_pairs = [
    ["homosexual", "heterosexual"],
    ["gay", "heterosexual"],
    ["lesbian", "heterosexual"],
    ["women","men"],
    # ["black", "white"]
]

for group_pair in group_pairs:

    g1 = group_pair[0]
    g2 = group_pair[1]

    rows = []

    results = plot_res[g1]
    for target_lm, scores in results.items():
        if target_lm not in plot_target_lms:
            continue

        row = [g1,target_lm]+scores
        rows.append(row)

    results = plot_res[g2]
    for target_lm, scores in results.items():
        if target_lm not in plot_target_lms:
            continue

        row = [g2,target_lm]+scores
        rows.append(row)

    df = pd.DataFrame(rows, columns=["group", "llm"]+red_lm_names)
    # display(df)

    g1_scores = df[df["group"]==g1][red_lm_names].to_numpy()
    # print(g1_scores.shape)

    g2_scores = df[df["group"]==g2][red_lm_names].to_numpy()
    # print(g2_scores.shape)

    diff = np.abs(g1_scores-g2_scores)
    # print(diff)

    rows = []
    for i, tllm in enumerate(plot_target_lms):
        row = ["diff",tllm]+diff[i].tolist()
        rows.append(row)
    new_df = pd.DataFrame(rows, columns=["group", "llm"]+red_lm_names)

    df = pd.concat([df, new_df])
    print("Final")
    display(df)
    print()
    print()

    df = df[df["group"] == "diff"]

    rank_df = pd.DataFrame()
    rank_df["llm"] = df["llm"].tolist()
    for red_lm in red_lm_list:
        scores = df[red_lm].tolist()
        ranks = np.argsort(scores).tolist()
        final_ranks = []
        for idx, r in enumerate(ranks):
            final_ranks.append(ranks.index(idx)+1)

        rank_df[red_lm] = final_ranks

    display(rank_df)
    print()
    print("-------------")